### Imports and data loading

In [81]:
import pandas as pd
import geopandas as gpd
import folium
import skmob
from shapely.geometry import LineString
from skmob.preprocessing.detection import stay_locations
from skmob.preprocessing.filtering import filter
from skmob.preprocessing.compression import compress
import numpy as np

In [28]:
# 6 cargo ships from Brest dataset 
data = pd.read_csv('data.csv')
data.t = pd.to_datetime(data.t, unit = 's')

### Data preprocessing

In [68]:
# Use skmob TrajDataFrame
data = skmob.TrajDataFrame(data, latitude='lat', longitude='lon', datetime='t', user_id='sourcemmsi', timestamp=True)

In [69]:
data = filter(data, speed_kmh=100)

In [70]:
# Compress data for more light weight LineStrings
cdata = compress(data, spatial_radius_km=0.2)

In [71]:
cdata = cdata.to_geodataframe()

/home/jussiste/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [76]:
cdata.set_crs(4326, allow_override=True,inplace=True)

,uid,navigationalstatus,rateofturn,speedoverground,courseoverground,trueheading,lng,lat,datetime,geometry
0,228020700,0,0,16.5,205.0,203,-5.918172,48.695835,2015-10-02 13:33:15,POINT (-5.91817 48.69584)
1,228020700,0,0,16.5,206.0,200,-5.919533,48.693910,2015-10-02 13:33:49,POINT (-5.91953 48.69391)
2,228020700,0,0,15.0,154.0,145,-5.908808,48.633940,2015-10-02 13:48:31,POINT (-5.90881 48.63394)
3,228020700,0,0,14.9,154.0,145,-5.907538,48.632195,2015-10-02 13:48:59,POINT (-5.90754 48.63220)
4,228020700,0,0,13.3,138.0,137,-5.442703,48.185074,2015-10-02 16:06:19,POINT (-5.44270 48.18507)
...,...,...,...,...,...,...,...,...,...,...
12944,538002973,0,0,12.3,319.0,318,-5.183365,48.310318,2015-12-11 23:04:31,POINT (-5.18337 48.31032)
12945,538002973,0,-126,12.1,321.0,312,-5.187465,48.313658,2015-12-11 23:05:31,POINT (-5.18746 48.31366)
12946,538002973,0,0,12.3,291.0,291,-5.206332,48.319466,2015-12-11 23:09:59,POINT (-5.20633 48.31947)
12947,538002973,0,0,12.6,291.0,291,-5.212957,48.321192,2015-12-11 23:11:10,POINT (-5.21296 48.32119)


In [77]:
cdata.sort_values(['uid', 'datetime'], inplace = True)

In [56]:
# Detect areas where the ships have stopped
stops = stay_locations(data, minutes_for_a_stop=120, spatial_radius_km=0.1)
stops.sort_values(['uid', 'datetime'], inplace = True)
stops['next_uid'] = stops.uid.shift(-1)
stops['next_time'] = stops.datetime.shift(-1)

In [78]:
trajs = []
for i, row in stops.iterrows():
    if row['next_uid']!=row['uid']:
        continue
    trajs.append(cdata[(cdata.uid == row.uid) & (cdata.datetime>row.datetime) & (cdata.datetime<=row.next_time)])

In [66]:
data = data.to_geodataframe()

/home/jussiste/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [83]:
def trajectories(trajs):
    t = []
    for traj in trajs:
        if len(traj)<=1:
            continue
        t.append(np.append(traj.iloc[0].values, [traj.iloc[len(traj)-1].datetime, traj.iloc[len(traj)-1].geometry, LineString(traj.geometry.values)]))
    return pd.DataFrame(t, columns=[*trajs[0].columns, "end_time", "end_coords", "trajectory"])

In [84]:
trajs_df = trajectories(trajs)

### Data mapping

In [87]:
# Initializing the map object
m = folium.Map(location=[stops.lat.mean(), stops.lng.mean()], 
                 zoom_start=10, control_scale=True)


In [88]:
m

In [102]:
### Adding trajectory data to map
def add_ship_trajectories(lines, m, feature_group, color):
    for _, r in lines.items():
        sim_geo = gpd.GeoSeries(r)
        geo_j = sim_geo.to_json()
        geo_j = folium.GeoJson(data=geo_j,
                               style_function=lambda x: {"opacity": 0.50, "color": color,}).add_to(feature_group)
    feature_group.add_to(m)

fg = folium.map.FeatureGroup(name='Trajectories', show = True)
add_ship_trajectories(trajs_df.trajectory, m, fg, 'green')
m

In [112]:
# Add separate trajectories for each ship
m = folium.Map(location=[stops.lat.mean(), stops.lng.mean()], 
                 zoom_start=10, control_scale=True)
colors = iter(['black', 'white', 'red', 'green', 'yellow', 'pink'])
for mmsi in trajs_df.uid.unique():
    fg = folium.map.FeatureGroup(name= f'trajectories for ship {mmsi}', show = True)
    add_ship_trajectories(trajs_df[trajs_df.uid==mmsi].trajectory, m, fg, next(colors))
#folium.LayerControl(collapsed=False).add_to(m)


In [113]:
m

In [114]:
### Add extra tile layers

# Satellite imagery
folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = False,
        control = True
       ).add_to(m)

# Ocean bathymetry
ocean = folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/Ocean/World_Ocean_Base/MapServer/tile/{z}/{y}/{x}',
        attr = 'Ocean',
        name = 'Ocean',
        overlay = False,
        control = True
       ).add_to(m)

# Ocean additional reference 

folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/Ocean/World_Ocean_Reference/MapServer/tile/{z}/{y}/{x}',
        attr = 'Ocean_ref',
        name = 'Ocean_ref',
        overlay = True,
        control = True
       ).add_to(m)

folium.LayerControl(collapsed=False).add_to(m)


More tile layers can be found at [https://server.arcgisonline.com/arcgis/rest/services]()

In [115]:
m

In [126]:
stops = stops.to_geodataframe()

/home/jussiste/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [131]:
pip freeze |grep folium

folium==0.14.0
Note: you may need to restart the kernel to use updated packages.


In [132]:
# Mapping point based data
m = folium.Map(location=[stops.lat.mean(), stops.lng.mean()], 
                 zoom_start=10, control_scale=True)

marker_cluster = folium.plugins.MarkerCluster().add_to(m)

for _, row in stops.iterrows():
    print(row)
    #folium.Marker(row.geometry).add_to(marker_cluster)

ValueError: could not convert string to float: 'S'